In [ ]:
!pip install flask pyngrok


In [ ]:
from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)

@app.route("/")
def home():
    return """
    <!DOCTYPE html>
    <html>
    <head>
        <title>GruhaBuddy AI</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background: linear-gradient(to right, #f8f9fa, #e9ecef);
                text-align: center;
                padding: 50px;
            }
            h1 {
                font-size: 40px;
                color: #333;
            }
            p {
                font-size: 20px;
                color: #555;
            }
            .btn {
                padding: 12px 25px;
                background-color: #007bff;
                color: white;
                border: none;
                border-radius: 10px;
                font-size: 18px;
                cursor: pointer;
            }
            .btn:hover {
                background-color: #0056b3;
            }
            .card {
                background: white;
                padding: 20px;
                margin: 20px auto;
                width: 300px;
                border-radius: 15px;
                box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
            }
        </style>
    </head>
    <body>
        <h1>🏠 GruhaBuddy – Intelligent Interior Design Assistant</h1>
        <p>AI-powered smart interior design suggestions for your dream home.</p>

        <div class="card">
            <h3>Smart Space Planning</h3>
            <p>Optimize room layout using AI.</p>
        </div>

        <div class="card">
            <h3>Personalized Designs</h3>
            <p>Get custom themes based on your taste.</p>
        </div>

        <div class="card">
            <h3>AI Chatbot">
            <p>Ask design questions instantly.</p>
        </div>

        <br>
        <button class="btn">Get Started</button>
    </body>
    </html>
    """

# IMPORTANT: Replace "YOUR_AUTHTOKEN" with your actual ngrok authtoken.
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("YOUR_AUTHTOKEN")

# Run app with ngrok
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(port=5000)

ERROR:pyngrok.process.ngrok:t=2026-02-13T10:02:17+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2026-02-13T10:02:17+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
    <title>GruhaBuddy AI</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(to right, #f0f2f5, #dfe9f3);
            text-align: center;
            padding: 50px;
        }
        h1 {
            font-size: 40px;
            color: #333;
        }
        p {
            font-size: 18px;
            color: #555;
            max-width: 600px;
            margin: auto;
        }
        .section {
            background: white;
            padding: 20px;
            margin: 20px auto;
            width: 60%;
            border-radius: 15px;
            box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
        }
        button {
            padding: 12px 25px;
            font-size: 16px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
        }
        button:hover {
            background: #45a049;
        }
    </style>
</head>
<body>

    <h1>🏠 GruhaBuddy</h1>
    <p>GruhaBuddy is an Intelligent Interior Design AI Assistant that helps users create beautiful and smart home interiors using AI-powered recommendations.</p>

    <div class="section">
        <h2>✨ Smart Space Planning</h2>
        <p>Optimize your room layout with AI-based suggestions.</p>
    </div>

    <div class="section">
        <h2>🎨 Personalized Design</h2>
        <p>Get customized interior themes based on your style and budget.</p>
    </div>

    <div class="section">
        <h2>🤖 AI Chat Support</h2>
        <p>Ask design-related questions and receive instant guidance.</p>
    </div>

    <br>
    <button>Get Started</button>

</body>
</html>
""")


In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
    <title>GruhaBuddy AI</title>

    <style>
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(to right, #f0f2f5, #dfe9f3);
            text-align: center;
            padding: 50px;
        }

        h1 {
            font-size: 40px;
            color: #333;
        }

        p {
            font-size: 18px;
            color: #555;
            max-width: 600px;
            margin: auto;
        }

        .section {
            background: white;
            padding: 20px;
            margin: 20px auto;
            width: 60%;
            border-radius: 15px;
            box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
        }

        button {
            padding: 12px 25px;
            font-size: 16px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
        }

        button:hover {
            background: #45a049;
        }
    </style>
</head>

<body>

    <h1>🏠 GruhaBuddy</h1>

    <p>
        GruhaBuddy is an Intelligent Interior Design AI Assistant
        that helps users create beautiful and smart home interiors
        using AI-powered recommendations.
    </p>

    <div class="section">
        <h2>✨ Smart Space Planning</h2>
        <p>Optimize your room layout with AI-based suggestions.</p>
    </div>

    <div class="section">
        <h2>🎨 Personalized Design</h2>
        <p>Get customized interior themes based on your style and budget.</p>
    </div>

    <div class="section">
        <h2>🤖 AI Chat Support</h2>
        <p>Ask design-related questions and receive instant guidance.</p>
    </div>

    <br>
    <button>Get Started</button>

</body>
</html>
""")

In [ ]:
from IPython.display import display, HTML

html_code = """
<!DOCTYPE html>
<html>
<head>
<title>GruhaBuddy AI</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: linear-gradient(to right, #f0f2f5, #dfe9f3);
    text-align: center;
    padding: 50px;
}

h1 {
    font-size: 40px;
    color: #333;
}

p {
    font-size: 18px;
    color: #555;
    max-width: 600px;
    margin: auto;
}

.section {
    background: white;
    padding: 20px;
    margin: 20px auto;
    width: 60%;
    border-radius: 15px;
    box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
}

button {
    padding: 12px 25px;
    font-size: 16px;
    background: #4CAF50;
    color: white;
    border: none;
    border-radius: 8px;
    cursor: pointer;
}

button:hover {
    background: #45a049;
}
</style>
</head>

<body>

<h1>🏠 GruhaBuddy</h1>

<p>
GruhaBuddy is an Intelligent Interior Design AI Assistant
that helps users create beautiful and smart home interiors
using AI-powered recommendations.
</p>

<div class="section">
<h2>✨ Smart Space Planning</h2>
<p>Optimize your room layout with AI-based suggestions.</p>
</div>

<div class="section">
<h2>🎨 Personalized Design</h2>
<p>Get customized interior themes based on your style and budget.</p>
</div>

<div class="section">
<h2>🤖 AI Chat Support</h2>
<p>Ask design-related questions and receive instant guidance.</p>
</div>

<br>
<button>Get Started</button>

</body>
</html>
"""

display(HTML(html_code))


In [ ]:
display(HTML(html_code))


In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>GruhaBuddy AI</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: linear-gradient(to right, #f0f2f5, #dfe9f3);
    text-align: center;
    padding: 50px;
}

h1 {
    font-size: 40px;
    color: #333;
}

p {
    font-size: 18px;
    color: #555;
    max-width: 600px;
    margin: auto;
}

.section {
    background: white;
    padding: 20px;
    margin: 20px auto;
    width: 60%;
    border-radius: 15px;
    box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
}

button {
    padding: 12px 25px;
    font-size: 16px;
    background: #4CAF50;
    color: white;
    border: none;
    border-radius: 8px;
    cursor: pointer;
}

button:hover {
    background: #45a049;
}

#appSection {
    display: none;
}
input {
    padding: 10px;
    width: 60%;
    margin-top: 15px;
}
</style>

<script>
function startApp() {
    document.getElementById("homeSection").style.display = "none";
    document.getElementById("appSection").style.display = "block";
}

function getSuggestion() {
    let userInput = document.getElementById("userInput").value;
    let response = "";

    if(userInput.toLowerCase().includes("bedroom")) {
        response = "Use soft lighting, pastel colors, and minimal furniture for a cozy bedroom.";
    }
    else if(userInput.toLowerCase().includes("hall")) {
        response = "Choose a modern sofa, wall art, and warm lights for a stylish hall.";
    }
    else {
        response = "Try neutral colors, smart storage, and indoor plants for better interiors.";
    }

    document.getElementById("response").innerHTML = response;
}
</script>

</head>

<body>

<!-- HOME SECTION -->
<div id="homeSection">

<h1>🏠 GruhaBuddy</h1>

<p>
GruhaBuddy is an Intelligent Interior Design AI Assistant
that helps users create beautiful and smart home interiors
using AI-powered recommendations.
</p>

<div class="section">
<h2>✨ Smart Space Planning</h2>
<p>Optimize your room layout with AI-based suggestions.</p>
</div>

<div class="section">
<h2>🎨 Personalized Design</h2>
<p>Get customized interior themes based on your style and budget.</p>
</div>

<div class="section">
<h2>🤖 AI Chat Support</h2>
<p>Ask design-related questions and receive instant guidance.</p>
</div>

<br>
<button onclick="startApp()">Get Started</button>

</div>

<!-- AI APP SECTION -->
<div id="appSection">

<h1>🤖 GruhaBuddy Design Assistant</h1>

<p>Type your room type (example: bedroom, hall, kitchen)</p>

<input type="text" id="userInput" placeholder="Enter room type">
<br><br>
<button onclick="getSuggestion()">Get Design Suggestion</button>

<div class="section">
<p id="response"></p>
</div>

</div>

</body>
</html>
""")

In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>GruhaBuddy AI Room Designer</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: linear-gradient(to right, #f0f2f5, #dfe9f3);
    text-align: center;
    padding: 30px;
}

h1 {
    font-size: 40px;
    color: #333;
}

.section {
    background: white;
    padding: 20px;
    margin: 20px auto;
    width: 70%;
    border-radius: 15px;
    box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
}

button {
    padding: 12px 25px;
    font-size: 16px;
    background: #4CAF50;
    color: white;
    border: none;
    border-radius: 8px;
    cursor: pointer;
}

button:hover {
    background: #45a049;
}

select, input {
    padding: 10px;
    margin: 10px;
    width: 60%;
}

#designSection {
    display: none;
}

#result {
    font-weight: bold;
    color: #333;
}
</style>

<script>
function startApp() {
    document.getElementById("homeSection").style.display = "none";
    document.getElementById("designSection").style.display = "block";
}

function generateDesign() {
    let room = document.getElementById("roomType").value;
    let style = document.getElementById("designStyle").value;
    let budget = document.getElementById("budget").value;

    let resultText = "AI Design Suggestion for your " + room + ":\n\n";

    if(style === "Modern") {
        resultText += "• Use neutral wall colors\n• Add LED strip lighting\n• Minimal furniture\n";
    }
    else if(style === "Traditional") {
        resultText += "• Wooden furniture\n• Warm lighting\n• Decorative wall patterns\n";
    }
    else if(style === "Minimalist") {
        resultText += "• Simple color palette\n• Space-saving furniture\n• Indoor plants\n";
    }

    resultText += "\nEstimated Budget Plan: ₹" + budget;

    document.getElementById("result").innerText = resultText;
}
</script>

</head>

<body>

<!-- HOME PAGE -->
<div id="homeSection">
    <h1>🏠 GruhaBuddy AI</h1>

    <div class="section">
        <p>
        Design your dream room using Artificial Intelligence.
        Choose your room type, style, and budget — GruhaBuddy will generate
        smart interior suggestions instantly.
        </p>
    </div>

    <button onclick="startApp()">Start Designing</button>
</div>


<!-- DESIGN PAGE -->
<div id="designSection">
    <h1>🤖 AI Room Designer</h1>

    <div class="section">
        <label>Select Room Type:</label><br>
        <select id="roomType">
            <option>Bedroom</option>
            <option>Hall</option>
            <option>Kitchen</option>
            <option>Office</option>
        </select><br>

        <label>Select Design Style:</label><br>
        <select id="designStyle">
            <option>Modern</option>
            <option>Traditional</option>
            <option>Minimalist</option>
        </select><br>

        <label>Enter Budget (₹):</label><br>
        <input type="number" id="budget" placeholder="Enter your budget"><br>

        <button onclick="generateDesign()">Generate AI Design</button>
    </div>

    <div class="section">
        <h3>✨ AI Generated Design Plan</h3>
        <p id="result"></p>
    </div>
</div>

</body>
</html>
""")

In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>GruhaBuddy AI – Intelligent Interior Designer</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: linear-gradient(to right, #e3f2fd, #fce4ec);
    text-align: center;
    padding: 30px;
}

h1 {
    font-size: 38px;
    color: #333;
}

.section {
    background: white;
    padding: 20px;
    margin: 20px auto;
    width: 75%;
    border-radius: 15px;
    box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
}

button {
    padding: 12px 25px;
    font-size: 16px;
    background: #4CAF50;
    color: white;
    border: none;
    border-radius: 8px;
    cursor: pointer;
}

button:hover {
    background: #388e3c;
}

select, input {
    padding: 10px;
    margin: 10px;
    width: 60%;
}

#designSection {
    display: none;
}

#result {
    font-weight: bold;
    white-space: pre-line;
}

#colorPreview {
    height: 80px;
    margin-top: 15px;
    border-radius: 10px;
}
</style>

<script>
function startApp() {
    document.getElementById("homeSection").style.display = "none";
    document.getElementById("designSection").style.display = "block";
}

function generateDesign() {

    let room = document.getElementById("roomType").value;
    let style = document.getElementById("designStyle").value;
    let budget = document.getElementById("budget").value;
    let color = document.getElementById("color").value;

    let resultText = "AI Design Plan for your " + room + ":\n\n";

    if(style === "Modern") {
        resultText += "• Neutral color palette\n";
        resultText += "• LED ceiling lighting\n";
        resultText += "• Minimal furniture setup\n";
    }
    else if(style === "Traditional") {
        resultText += "• Wooden furniture\n";
        resultText += "• Warm yellow lighting\n";
        resultText += "• Decorative wall frames\n";
    }
    else if(style === "Minimalist") {
        resultText += "• Simple wall colors\n";
        resultText += "• Space-saving storage\n";
        resultText += "• Indoor plants\n";
    }

    let estimatedCost = budget ? budget : "Not specified";

    resultText += "\nSelected Wall Color: " + color;
    resultText += "\nEstimated Budget: ₹" + estimatedCost;

    document.getElementById("result").innerText = resultText;

    // Show color preview
    document.getElementById("colorPreview").style.backgroundColor = color;
}
</script>

</head>

<body>

<!-- HOME SECTION -->
<div id="homeSection">

<h1>🏠 GruhaBuddy AI</h1>

<div class="section">
<p>
GruhaBuddy is an Intelligent Interior Design Assistant.
Design your dream room using AI-powered suggestions.
Select your room, style, color, and budget — get instant design plans!
</p>
</div>

<button onclick="startApp()">Start Designing</button>

</div>


<!-- DESIGN SECTION -->
<div id="designSection">

<h1>🤖 AI Room Designer</h1>

<div class="section">

<label>Select Room Type:</label><br>
<select id="roomType">
<option>Bedroom</option>
<option>Hall</option>
<option>Kitchen</option>
<option>Office</option>
</select><br>

<label>Select Design Style:</label><br>
<select id="designStyle">
<option>Modern</option>
<option>Traditional</option>
<option>Minimalist</option>
</select><br>

<label>Select Wall Color:</label><br>
<input type="color" id="color" value="#ffffff"><br>

<label>Enter Budget (₹):</label><br>
<input type="number" id="budget" placeholder="Enter your budget"><br>

<button onclick="generateDesign()">Generate AI Design</button>

</div>

<div class="section">
<h3>✨ AI Generated Interior Plan</h3>
<p id="result"></p>
<div id="colorPreview"></div>
</div>

</div>

</body>
</html>
""")

In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>GruhaBuddy AI – Intelligent Interior Designer</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: linear-gradient(to right, #e3f2fd, #fce4ec);
    text-align: center;
    padding: 30px;
}

h1 {
    font-size: 38px;
    color: #333;
}

.section {
    background: white;
    padding: 20px;
    margin: 20px auto;
    width: 75%;
    border-radius: 15px;
    box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
}

button {
    padding: 12px 25px;
    font-size: 16px;
    background: #4CAF50;
    color: white;
    border: none;
    border-radius: 8px;
    cursor: pointer;
}

button:hover {
    background: #388e3c;
}

select, input, textarea {
    padding: 10px;
    margin: 10px;
    width: 60%;
}

#designSection {
    display: none;
}

#result {
    font-weight: bold;
    white-space: pre-line;
}
</style>

<script>
function startApp() {
    document.getElementById("homeSection").style.display = "none";
    document.getElementById("designSection").style.display = "block";
}

function generateDesign() {

    let room = document.getElementById("roomType").value;
    let style = document.getElementById("designStyle").value;
    let budget = document.getElementById("budget").value;
    let idea = document.getElementById("idea").value;

    let resultText = "AI Design Plan for your " + room + ":\n\n";

    if(style === "Modern") {
        resultText += "• Clean neutral colors\n";
        resultText += "• LED ceiling lights\n";
        resultText += "• Minimal furniture\n";
    }
    else if(style === "Traditional") {
        resultText += "• Wooden furniture\n";
        resultText += "• Warm lighting\n";
        resultText += "• Decorative frames\n";
    }
    else if(style === "Minimalist") {
        resultText += "• Simple wall tones\n";
        resultText += "• Space-saving storage\n";
        resultText += "• Indoor plants\n";
    }

    if(idea !== "") {
        resultText += "\nYour Custom Idea Included:\n• " + idea + "\n";
    }

    resultText += "\nEstimated Budget: ₹" + (budget ? budget : "Not specified");

    document.getElementById("result").innerText = resultText;
}
</script>

</head>

<body>

<!-- HOME SECTION -->
<div id="homeSection">

<h1>🏠 GruhaBuddy AI</h1>

<div class="section">
<p>
Design your dream room using AI-powered suggestions.
Click below to start designing your room with your own creative ideas.
</p>
</div>

<button onclick="startApp()">Start Designing</button>

</div>


<!-- DESIGN SECTION -->
<div id="designSection">

<h1>🤖 AI Room Designer</h1>

<div class="section">

<label>Select Room Type:</label><br>
<select id="roomType">
<option>Bedroom</option>
<option>Hall</option>
<option>Kitchen</option>
<option>Office</option>
</select><br>

<label>Select Design Style:</label><br>
<select id="designStyle">
<option>Modern</option>
<option>Traditional</option>
<option>Minimalist</option>
</select><br>

<label>Enter Your Own Design Idea:</label><br>
<textarea id="idea" rows="3" placeholder="Example: Add blue sofa and wooden shelves"></textarea><br>

<label>Enter Budget (₹):</label><br>
<input type="number" id="budget" placeholder="Enter your budget"><br>

<button onclick="generateDesign()">Generate AI Design</button>

</div>

<div class="section">
<h3>✨ AI Generated Interior Plan</h3>
<p id="result"></p>
</div>

</div>

</body>
</html>
""")

In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
    <title>GruhaBuddy AI – Intelligent Interior Designer</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(to right, #e3f2fd, #fce4ec);
            text-align: center;
            padding: 30px;
        }

        h1 {
            font-size: 38px;
            color: #333;
        }

        .section {
            background: white;
            padding: 20px;
            margin: 20px auto;
            width: 75%;
            border-radius: 15px;
            box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
        }

        button {
            padding: 12px 25px;
            font-size: 16px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
        }

        button:hover {
            background: #388e3c;
        }

        select, input, textarea {
            padding: 10px;
            margin: 10px;
            width: 60%;
        }

        #designSection {
            display: none;
        }

        #result {
            font-weight: bold;
            white-space: pre-line;
            margin-top: 20px;
        }
    </style>
</head>

<body>

<!-- HOME SECTION -->
<div id="homeSection" class="section">
    <h1>🏠 GruhaBuddy AI</h1>
    <p>Design your dream room using AI-powered suggestions.</p>
    <p>Click below to start designing your room with your own creative ideas.</p>
    <button onclick="startApp()">Start Designing</button>
</div>

<!-- DESIGN SECTION -->
<div id="designSection" class="section">
    <h2>🤖 AI Room Designer</h2>

    <p>Select Room Type:</p>
    <select id="roomType">
        <option>Bedroom</option>
        <option>Hall</option>
        <option>Kitchen</option>
        <option>Office</option>
    </select>

    <p>Select Design Style:</p>
    <select id="designStyle">
        <option>Modern</option>
        <option>Traditional</option>
        <option>Minimalist</option>
    </select>

    <p>Enter Your Own Design Idea:</p>
    <textarea id="idea" placeholder="Example: Add a study table near window"></textarea>

    <p>Enter Budget (₹):</p>
    <input type="number" id="budget" placeholder="Enter budget">

    <br><br>
    <button onclick="generateDesign()">Generate AI Design</button>
    <button onclick="goBack()">Back</button>

    <div id="result"></div>
</div>

<script>
function startApp() {
    document.getElementById("homeSection").style.display = "none";
    document.getElementById("designSection").style.display = "block";
}

function goBack() {
    document.getElementById("designSection").style.display = "none";
    document.getElementById("homeSection").style.display = "block";
}

function generateDesign() {

    let room = document.getElementById("roomType").value;
    let style = document.getElementById("designStyle").value;
    let budget = document.getElementById("budget").value;
    let idea = document.getElementById("idea").value;

    let resultText = "✨ AI Design Plan for your " + room + ":\n\n";

    if(style === "Modern") {
        resultText += "• Clean neutral colors\n";
        resultText += "• LED ceiling lights\n";
        resultText += "• Minimal furniture\n";
    }
    else if(style === "Traditional") {
        resultText += "• Wooden furniture\n";
        resultText += "• Warm lighting\n";
        resultText += "• Decorative frames\n";
    }
    else if(style === "Minimalist") {
        resultText += "• Simple wall tones\n";
        resultText += "• Space-saving storage\n";
        resultText += "• Indoor plants\n";
    }

    if(idea !== "") {
        resultText += "\nYour Custom Idea Included:\n• " + idea + "\n";
    }

    resultText += "\nEstimated Budget: ₹" + (budget ? budget : "Not specified");

    document.getElementById("result").innerText = resultText;
}
</script>

</body>
</html>
""")

In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
    <title>GruhaBuddy AI – Intelligent Interior Designer</title>

    <style>
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(to right, #e3f2fd, #fce4ec);
            text-align: center;
            padding: 30px;
        }

        h1 {
            font-size: 38px;
            color: #333;
        }

        .section {
            background: white;
            padding: 20px;
            margin: 20px auto;
            width: 75%;
            border-radius: 15px;
            box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
        }

        button {
            padding: 12px 25px;
            font-size: 16px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
        }

        button:hover {
            background: #388e3c;
        }

        select, input, textarea {
            padding: 10px;
            margin: 10px;
            width: 60%;
        }

        #designSection {
            display: none;
        }

        #result {
            font-weight: bold;
            white-space: pre-line;
            margin-top: 20px;
        }
    </style>
</head>

<body>

<!-- HOME SECTION -->
<div id="homeSection" class="section">
    <h1>🏠 GruhaBuddy AI</h1>
    <p>Design your dream room using AI-powered suggestions.</p>
    <p>Click below to start designing your room with your own creative ideas.</p>
    <button onclick="startApp()">Start Designing</button>
</div>

<!-- DESIGN SECTION -->
<div id="designSection" class="section">
    <h2>🤖 AI Room Designer</h2>

    <p>Select Room Type:</p>
    <select id="roomType">
        <option>Bedroom</option>
        <option>Hall</option>
        <option>Kitchen</option>
        <option>Office</option>
    </select>

    <p>Select Design Style:</p>
    <select id="designStyle">
        <option>Modern</option>
        <option>Traditional</option>
        <option>Minimalist</option>
    </select>

    <p>Enter Your Own Design Idea:</p>
    <textarea id="idea" placeholder="Example: Add study table near window"></textarea>

    <p>Enter Budget (₹):</p>
    <input type="number" id="budget" placeholder="Enter budget">

    <br><br>
    <button onclick="generateDesign()">Generate AI Design</button>
    <button onclick="goBack()">Back</button>

    <div id="result"></div>
</div>

<script>
function startApp() {
    document.getElementById("homeSection").style.display = "none";
    document.getElementById("designSection").style.display = "block";
}

function goBack() {
    document.getElementById("designSection").style.display = "none";
    document.getElementById("homeSection").style.display = "block";
}

function generateDesign() {

    let room = document.getElementById("roomType").value;
    let style = document.getElementById("designStyle").value;
    let budget = document.getElementById("budget").value;
    let idea = document.getElementById("idea").value;

    let resultText = "✨ AI Design Plan for your " + room + ":\n\n";

    if(style === "Modern") {
        resultText += "• Clean neutral colors\n";
        resultText += "• LED ceiling lights\n";
        resultText += "• Minimal furniture\n";
    }
    else if(style === "Traditional") {
        resultText += "• Wooden furniture\n";
        resultText += "• Warm lighting\n";
        resultText += "• Decorative frames\n";
    }
    else if(style === "Minimalist") {
        resultText += "• Simple wall tones\n";
        resultText += "• Space-saving storage\n";
        resultText += "• Indoor plants\n";
    }

    if(idea !== "") {
        resultText += "\nYour Custom Idea Included:\n• " + idea + "\n";
    }

    resultText += "\nEstimated Budget: ₹" + (budget ? budget : "Not specified");

    document.getElementById("result").innerText = resultText;
}
</script>

</body>
</html>
""")

In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
    <title>GruhaBuddy AI – Intelligent Interior Designer</title>

    <style>
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(to right, #e3f2fd, #fce4ec);
            text-align: center;
            padding: 30px;
        }

        h1 {
            font-size: 38px;
            color: #333;
        }

        .section {
            background: white;
            padding: 20px;
            margin: 20px auto;
            width: 75%;
            border-radius: 15px;
            box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
        }

        button {
            padding: 12px 25px;
            font-size: 16px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
        }

        button:hover {
            background: #388e3c;
        }

        select, input, textarea {
            padding: 10px;
            margin: 10px;
            width: 60%;
        }

        #designSection {
            display: none;
        }

        #result {
            font-weight: bold;
            white-space: pre-line;
            margin-top: 20px;
        }
    </style>
</head>

<body>

<!-- HOME SECTION -->
<div id="homeSection" class="section">
    <h1>🏠 GruhaBuddy AI</h1>
    <p>Design your dream room using AI-powered suggestions.</p>
    <p>Click below to start designing your room with your own creative ideas.</p>
    <button onclick="startApp()">Start Designing</button>
</div>

<!-- DESIGN SECTION -->
<div id="designSection" class="section">
    <h2>🤖 AI Room Designer</h2>

    <p>Select Room Type:</p>
    <select id="roomType">
        <option>Bedroom</option>
        <option>Hall</option>
        <option>Kitchen</option>
        <option>Office</option>
    </select>

    <p>Select Design Style:</p>
    <select id="designStyle">
        <option>Modern</option>
        <option>Traditional</option>
        <option>Minimalist</option>
    </select>

    <p>Enter Your Own Design Idea:</p>
    <textarea id="idea" placeholder="Example: Add study table near window"></textarea>

    <p>Enter Budget (₹):</p>
    <input type="number" id="budget" placeholder="Enter budget">

    <br><br>
    <button onclick="generateDesign()">Generate AI Design</button>
    <button onclick="goBack()">Back</button>

    <div id="result"></div>
</div>

<script>
function startApp() {
    document.getElementById("homeSection").style.display = "none";
    document.getElementById("designSection").style.display = "block";
}

function goBack() {
    document.getElementById("designSection").style.display = "none";
    document.getElementById("homeSection").style.display = "block";
}

function generateDesign() {

    let room = document.getElementById("roomType").value;
    let style = document.getElementById("designStyle").value;
    let budget = document.getElementById("budget").value;
    let idea = document.getElementById("idea").value;

    let resultText = "✨ AI Design Plan for your " + room + ":\n\n";

    if(style === "Modern") {
        resultText += "• Clean neutral colors\n";
        resultText += "• LED ceiling lights\n";
        resultText += "• Minimal furniture\n";
    }
    else if(style === "Traditional") {
        resultText += "• Wooden furniture\n";
        resultText += "• Warm lighting\n";
        resultText += "• Decorative frames\n";
    }
    else if(style === "Minimalist") {
        resultText += "• Simple wall tones\n";
        resultText += "• Space-saving storage\n";
        resultText += "• Indoor plants\n";
    }

    if(idea !== "") {
        resultText += "\nYour Custom Idea Included:\n• " + idea + "\n";
    }

    resultText += "\nEstimated Budget: ₹" + (budget ? budget : "Not specified");

    document.getElementById("result").innerText = resultText;
}
</script>

</body>
</html>
""")

In [ ]:
# Install Flask
!pip install flask-ngrok -q

from flask import Flask, render_template_string, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

html_code = """
<!DOCTYPE html>
<html>
<head>
    <title>GruhaBuddy AI – Intelligent Interior Designer</title>
    <style>
        body {
            font-family: Arial;
            background: linear-gradient(to right, #e3f2fd, #fce4ec);
            text-align: center;
            padding: 40px;
        }
        .box {
            background: white;
            padding: 25px;
            margin: auto;
            width: 60%;
            border-radius: 15px;
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        }
        button {
            padding: 10px 20px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
        }
        select, input, textarea {
            padding: 10px;
            margin: 10px;
            width: 80%;
        }
    </style>
</head>
<body>

<div class="box">
<h1>🏠 GruhaBuddy AI</h1>
<p>Intelligent Interior Design Assistant</p>

<form method="POST">
<select name="room">
<option>Bedroom</option>
<option>Hall</option>
<option>Kitchen</option>
<option>Office</option>
</select>

<br>

<select name="style">
<option>Modern</option>
<option>Traditional</option>
<option>Minimalist</option>
</select>

<br>

<textarea name="idea" placeholder="Enter your own idea"></textarea>

<br>

<input type="number" name="budget" placeholder="Enter Budget ₹">

<br><br>

<button type="submit">Generate AI Design</button>
</form>

{% if result %}
<h3>{{ result }}</h3>
{% endif %}

</div>

</body>
</html>
"""

@app.route("/", methods=["GET","POST"])
def home():
    result = ""
    if request.method == "POST":
        room = request.form["room"]
        style = request.form["style"]
        idea = request.form["idea"]
        budget = request.form["budget"]

        result = f"✨ AI Plan for {room}:\n"

        if style == "Modern":
            result += "• Neutral colors\n• LED lights\n• Minimal furniture\n"
        elif style == "Traditional":
            result += "• Wooden furniture\n• Warm lighting\n• Decorative frames\n"
        elif style == "Minimalist":
            result += "• Simple design\n• Space-saving storage\n• Indoor plants\n"

        if idea:
            result += f"\nYour Idea Included: {idea}\n"

        result += f"\nEstimated Budget: ₹{budget if budget else 'Not specified'}"

    return render_template_string(html_code, result=result)

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [ ]:
from IPython.display import display, HTML

html_code = """
<!DOCTYPE html>
<html>
<head>
    <style>
        body {
            font-family: Arial;
            background: linear-gradient(to right, #e3f2fd, #fce4ec);
            text-align: center;
            padding: 40px;
        }
        .box {
            background: white;
            padding: 25px;
            margin: auto;
            width: 60%;
            border-radius: 15px;
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        }
        button {
            padding: 10px 20px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
        }
        select, input, textarea {
            padding: 10px;
            margin: 10px;
            width: 80%;
        }
        #result {
            margin-top: 20px;
            font-weight: bold;
            white-space: pre-line;
        }
    </style>
</head>
<body>

<div class="box">
<h1>🏠 GruhaBuddy AI</h1>
<p>Intelligent Interior Design Assistant</p>

<select id="room">
<option>Bedroom</option>
<option>Hall</option>
<option>Kitchen</option>
<option>Office</option>
</select><br>

<select id="style">
<option>Modern</option>
<option>Traditional</option>
<option>Minimalist</option>
</select><br>

<textarea id="idea" placeholder="Enter your own idea"></textarea><br>

<input type="number" id="budget" placeholder="Enter Budget ₹"><br><br>

<button onclick="generateDesign()">Generate AI Design</button>

<div id="result"></div>

</div>

<script>
function generateDesign() {

    let room = document.getElementById("room").value;
    let style = document.getElementById("style").value;
    let idea = document.getElementById("idea").value;
    let budget = document.getElementById("budget").value;

    let result = "✨ AI Plan for " + room + "\\n\\n";

    if(style === "Modern") {
        result += "• Neutral colors\\n• LED lights\\n• Minimal furniture\\n";
    }
    else if(style === "Traditional") {
        result += "• Wooden furniture\\n• Warm lighting\\n• Decorative frames\\n";
    }
    else if(style === "Minimalist") {
        result += "• Simple design\\n• Space-saving storage\\n• Indoor plants\\n";
    }

    if(idea !== "") {
        result += "\\nYour Idea Included: " + idea + "\\n";
    }

    result += "\\nEstimated Budget: ₹" + (budget ? budget : "Not specified");

    document.getElementById("result").innerText = result;
}
</script>

</body>
</html>
"""

display(HTML(html_code))


In [ ]:
YOUR_OPENAI_API_KEY


In [ ]:
YOUR_OPENAI_API_KEY


In [ ]:
index.html



In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
    <title>GruhaBuddy AI</title>
    <style>
        body {
            font-family: Arial;
            background: linear-gradient(to right, #e3f2fd, #fce4ec);
            text-align: center;
            padding: 40px;
        }
        .box {
            background: white;
            padding: 25px;
            margin: auto;
            width: 60%;
            border-radius: 15px;
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        }
        button {
            padding: 12px 25px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-size: 16px;
        }
        select, input, textarea {
            padding: 10px;
            margin: 10px;
            width: 80%;
        }
        #result {
            margin-top: 20px;
            white-space: pre-line;
            text-align: left;
        }
        img {
            margin-top: 20px;
            width: 80%;
            border-radius: 12px;
        }
    </style>
</head>
<body>

<div class="box">
    <h1>🏠 GruhaBuddy AI</h1>
    <h3>Intelligent Interior Design Assistant</h3>

    <select id="room">
        <option>Bedroom</option>
        <option>Hall</option>
        <option>Kitchen</option>
        <option>Office</option>
    </select>

    <select id="style">
        <option>Modern</option>
        <option>Traditional</option>
        <option>Minimalist</option>
        <option>Luxury</option>
    </select>

    <textarea id="idea" placeholder="Enter your custom idea..."></textarea>
    <input type="number" id="budget" placeholder="Enter Budget (₹)">

    <br>
    <button onclick="generateDesign()">Generate AI Design</button>

    <div id="result"></div>
    <img id="designImage" />
</div>

<script>
async function generateDesign() {

    const apiKey = "YOUR_OPENAI_API_KEY";

    let room = document.getElementById("room").value;
    let style = document.getElementById("style").value;
    let idea = document.getElementById("idea").value;
    let budget = document.getElementById("budget").value;

    document.getElementById("result").innerText = "🤖 AI is designing your space...";
    document.getElementById("designImage").src = "";

    let prompt = `Create a professional interior design plan for a ${room} in ${style} style.\n    Include furniture placement, lighting, color theme and budget estimation in Indian Rupees.\n    User custom idea: ${idea}.\n    Budget limit: ${budget}`;

    // 1️⃣ Generate Text Design Plan
    const textResponse = await fetch("https://api.openai.com/v1/chat/completions", {
        method: "POST",
        headers: {
            "Content-Type": "application/json",
            "Authorization": "Bearer " + apiKey
        },
        body: JSON.stringify({
            model: "gpt-4o-mini",
            messages: [
                { role: "user", content: prompt }
            ],
            max_tokens: 500
        })
    });

    const textData = await textResponse.json();
    let designText = textData.choices[0].message.content;

    document.getElementById("result").innerText = designText;

    // 2️⃣ Generate AI Interior Image
    const imageResponse = await fetch("https://api.openai.com/v1/images/generations", {
        method: "POST",
        headers: {
            "Content-Type": "application/json",
            "Authorization": "Bearer " + apiKey
        },
        body: JSON.stringify({
            model: "dall-e-3",
            prompt: `${style} ${room} interior design, ultra realistic, 4k, professional photography`,
            size: "1024x1024"
        })
    });

    const imageData = await imageResponse.json();
    document.getElementById("designImage").src = imageData.data[0].url;
}
</script>

</body>
</html>
""")

In [ ]:
server.js
index.html


NameError: name 'server' is not defined